# Lung and Colon Cancer Detection
This notebook demonstrates the process of building a machine learning model to detect lung and colon cancer using histopathological images. The dataset contains labeled images of cancerous and non-cancerous (<em>healthy</em>) tissues. There are five classes in this dataset: 
- Lung benign tissue (<em>healthy</em>)
- Lung adenocarcinoma
- Lung squamos cell carcinoma
- Colon adenocarcinoma
- Colong benign tissue (<em>healthy</em>)

The goal is to compare different Convolutional Neural Networks (CNNs) to explore the strengths and weaknesses of various architectures and understand which ones perform best for the chosen dataset. Ultimately, the most robust classifier (CNN) will be identified and can accurately identify cancerous lung or colon tissues from the given sample images. For a fair comparison of the different CNNs, it is necessary to set some guidelines / rules:
- The dataset has to be properly preprocessed.
- The same training parameters are used 
  - Learning rate
  - Batch size
  - Number of epochs
- The same optimizer is used
  - Isolates the effect if the CNN architecture on performance

Those rules will ensure that the comparison is consistent, controlled and fair.

The dataset used in this notebook is sourced from Kaggle: https://www.kaggle.com/datasets/andrewmvd/lung-and-colon-cancer-histopathological-images/data

## Required Packages

In [20]:
import os
from sklearn.model_selection import train_test_split


## Dataset handling
### Import Dataset
The dataset containes 25.000 images.

In [21]:
lung_dataset = '../lung_colon_image_set/lung_image_sets'
colon_dataset = '../lung_colon_image_set/colon_image_sets'

if not os.path.exists(lung_dataset):
    raise FileNotFoundError(f"Dataset path '{lung_dataset}' does not exist!")
if not os.path.exists(colon_dataset):
    raise FileNotFoundError(f"Dataset path '{colon_dataset}' does not exist!")

lung_classes = sorted(os.listdir(lung_dataset))
colon_classes = sorted(os.listdir(colon_dataset))

print(f"Classes found in lung dataset: {lung_classes}")
print(f"Classes found in colon dataset: {colon_classes}")

lc_classes = sorted(set(lung_classes + colon_classes))
print(f"Lung and Colon classes combined: {lc_classes}")

Classes found in lung dataset: ['lung_aca', 'lung_n', 'lung_scc']
Classes found in colon dataset: ['colon_aca', 'colon_n']
Lung and Colon classes combined: ['colon_aca', 'colon_n', 'lung_aca', 'lung_n', 'lung_scc']


### Split Dataset
The dataset is split into the following three categories with pre defined percentages:
- Training data (<em>80 %</em>)
- Validation data (<em>10 %</em>)
- Testing data (<em>10 %</em>)

In [28]:
def prepare_splits(data_directories, split_ratios=(0.8, 0.1, 0.1)):
    all_data = {}
    for data_directory in data_directories:
        for class_name in sorted(os.listdir(data_directory)):
            class_directory = os.path.join(data_directory, class_name)
            if os.path.isdir(class_directory):
                all_data.setdefault(class_name, []).extend(os.path.join(class_directory, file_name) for file_name in os.listdir(class_directory))

    for class_name, files in all_data.items():
        print(f"Class '{class_name}' has {len(files)} images")

    dataset_splits = {'training': [], 'validation': [], 'testing': []}

    for class_name, files in all_data.items():
        training_dataset, temporary_dataset = train_test_split(files, test_size=(1 - split_ratios[0]), random_state=42)

        validation_dataset, testing_dataset = train_test_split(temporary_dataset, test_size=split_ratios[2]/(split_ratios[1] + split_ratios[2]), random_state=42)

        dataset_splits['training'].extend(training_dataset)
        dataset_splits['validation'].extend(validation_dataset)
        dataset_splits['testing'].extend(testing_dataset)

    return dataset_splits

dataset_directories = [lung_dataset, colon_dataset]
dataset_splits = prepare_splits(dataset_directories)

print(f"Training dataset has {len(dataset_splits['training'])} images")
print(f"Validation dataset has {len(dataset_splits['validation'])} images")
print(f"Testing dataset has {len(dataset_splits['testing'])} images")

Class 'lung_aca' has 5000 images
Class 'lung_n' has 5000 images
Class 'lung_scc' has 5000 images
Class 'colon_aca' has 5000 images
Class 'colon_n' has 5000 images
Training dataset has 20000 images
Validation dataset has 2500 images
Testing dataset has 2500 images


### Dataset Class
Initializes the dataset class.

In [ ]:
class CancerDetectionDataset(Dataset):
    